In [1]:
import sys
import time
sys.path.append('../shared')
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import copy
import elbo_calculator as ec
import train
import argparse
import os
import model_hi_vae as mhv
from scipy.io import loadmat
%load_ext autoreload

In [2]:
column_types=[0,0,1,3,2,0]
input_dim = 9
hidden_dim =3
latent_dim=4
s_dim=2
x_dim = 9

In [3]:
rows = 100

In [4]:
x0 = np.random.normal(1,2,size=(rows,1))
x1 = np.random.normal(0,1,size=(rows,1))
x2 = np.log(np.random.poisson(1.5,size=(rows,1))+ 0.01)
x3 = tf.one_hot(np.random.randint(0,3,size=rows),3).numpy()
x4 = tf.one_hot(np.random.randint(0,2,size=rows),2).numpy()
x5 = np.random.normal(0.5,1,size=(rows,1))

In [5]:
x = tf.concat([x0,x1,x2,x3,x4,x5], axis=-1).numpy().astype(np.float32)

In [6]:
x[0]

array([-1.8695854 ,  0.09637402,  0.00995033,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        , -1.2689085 ], dtype=float32)

In [7]:
model = mhv.model_hi_vae(input_dim, hidden_dim, latent_dim, s_dim, column_types)

In [8]:
z = model.encoder(x)

In [9]:
x.dtype

dtype('float32')

In [10]:
x.shape

(100, 9)

In [11]:
z[0]

<tf.Tensor: id=1338, shape=(100, 2), dtype=float32, numpy=
array([[0.42939028, 0.5706097 ],
       [0.62988555, 0.37011442],
       [0.594176  , 0.405824  ],
       [0.5727726 , 0.42722732],
       [0.14362004, 0.8563799 ],
       [0.20907769, 0.79092234],
       [0.26757655, 0.7324235 ],
       [0.88996625, 0.11003375],
       [0.56301326, 0.43698674],
       [0.79630727, 0.20369276],
       [0.84383404, 0.15616594],
       [0.11803108, 0.88196886],
       [0.10877135, 0.8912287 ],
       [0.45964453, 0.54035556],
       [0.26242858, 0.7375714 ],
       [0.88117933, 0.11882069],
       [0.55584216, 0.44415778],
       [0.12969656, 0.87030345],
       [0.5087049 , 0.49129516],
       [0.33565453, 0.6643455 ],
       [0.64291114, 0.35708883],
       [0.5543483 , 0.44565165],
       [0.35295025, 0.6470498 ],
       [0.2775429 , 0.72245705],
       [0.8467135 , 0.15328652],
       [0.11316548, 0.88683444],
       [0.10863339, 0.89136654],
       [0.1960532 , 0.80394673],
       [0.8661362

In [12]:
latent_dim,s_dim

(4, 2)

In [13]:
z[1].shape, z[2].shape

(TensorShape([2, 100, 4]), TensorShape([2, 100, 4]))

In [14]:
gen_z_func =model.get_z_generator()

In [15]:
zs =gen_z_func(x)

In [16]:
len(zs)

4

In [17]:
zs[0]

<tf.Tensor: id=1400, shape=(100, 2), dtype=float32, numpy=
array([[0.42939028, 0.5706097 ],
       [0.62988555, 0.37011442],
       [0.594176  , 0.405824  ],
       [0.5727726 , 0.42722732],
       [0.14362004, 0.8563799 ],
       [0.20907769, 0.79092234],
       [0.26757655, 0.7324235 ],
       [0.88996625, 0.11003375],
       [0.56301326, 0.43698674],
       [0.79630727, 0.20369276],
       [0.84383404, 0.15616594],
       [0.11803108, 0.88196886],
       [0.10877135, 0.8912287 ],
       [0.45964453, 0.54035556],
       [0.26242858, 0.7375714 ],
       [0.88117933, 0.11882069],
       [0.55584216, 0.44415778],
       [0.12969656, 0.87030345],
       [0.5087049 , 0.49129516],
       [0.33565453, 0.6643455 ],
       [0.64291114, 0.35708883],
       [0.5543483 , 0.44565165],
       [0.35295025, 0.6470498 ],
       [0.2775429 , 0.72245705],
       [0.8467135 , 0.15328652],
       [0.11316548, 0.88683444],
       [0.10863339, 0.89136654],
       [0.1960532 , 0.80394673],
       [0.8661362

In [18]:
s_prop, beta, gamma, zs= zs

In [19]:
output = model.decoder([
                tf.reshape(zs,(-1, zs.shape[-1])),
                s_prop,
                beta,
                gamma])

ValueError: Dimensions must be equal, but are 2 and 100 for 'add' (op: 'AddV2') with input shapes: [20000,2,1], [100,2].

In [ ]:
y = np.random.normal(0,1,(2,3))

In [ ]:
id_mat = tf.stack([tf.eye(s_dim)], axis=0)
id_mat_batch = tf.tile(id_mat, tf.stack([tf.shape(y)[0], 1, 1]))

In [ ]:
id_mat_batch

In [ ]:
x_s = np.random.normal(0,2,(3,4))
id_mat1 = tf.stack([tf.eye(s_dim)], axis=0)
id_mat_batch1 = tf.tile(id_mat1, tf.stack([tf.shape(x_s)[0], 1, 1]))

In [ ]:
tf.stack([2, 1, 1])

In [ ]:
tf.tile(id_mat, [3, 1, 1])

In [ ]:
tf.tile(id_mat, tf.stack([3, 1, 1]))

In [ ]:
def get_encoder1(s_dim, x_dim):
    x = tf.keras.layers.Input(shape=(x_dim,))
    id_mat = tf.stack([tf.eye(s_dim)], axis=0)
    id_mat_batch = tf.tile(id_mat, tf.stack([tf.shape(x)[0], 1, 1]))
    x_s = tf.keras.backend.repeat(x,s_dim)
    x_s = tf.concat([x_s, id_mat_batch], axis=-1)
    return keras.models.Model(
        inputs=x,
        outputs=x_s
    )

def get_encoder2(s_dim, x_dim):
    x = tf.keras.layers.Input(shape=(x_dim,))
    id_mat = tf.stack([tf.eye(s_dim)], axis=0)
    id_mat_batch = tf.tile(id_mat, tf.stack([tf.shape(x)[0], 1, 1]))
    x_s = tf.keras.backend.repeat(x,s_dim)
    x_s = tf.concat([x_s, id_mat_batch], axis=-1)
    return keras.models.Model(
        inputs=x,
        outputs=x_s
    )

In [ ]:
ecd1 = get_encoder1(s_dim,x_dim)
ecd2 = get_encoder2(s_dim,x_dim)